# Project Overview

You receive a European Soccer Database that has more than 25,000 matches and more than 10,000 players for European professional soccer seasons from 2008 to 2016. The goal is you walk through this database to do analysis include some steps for exploring our dataset, some steps for basics statistics and then you visualize the result. To complete all your steps, you need to query your data in the database using SQL statement. This project practices you write SQL command to pull data and extrac it. 

# Database Description

This European Soccer Database has more than 25,000 matches and more than 10,000 players for European professional soccer seasons from 2008 to 2016, 11 European Countries with their lead championship. Players and Teams' attributes sourced from EA Sports' FIFA video game series, including the weekly updates.

# Import Python package

In [1]:
#Import libraries
import numpy as np
import pandas as pd
import sqlite3
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 100)

## Question 1: Connect to database and get information of all tables

Read https://docs.python.org/2/library/sqlite3.html

You write only SQL statement to get the result, should not use Pandas to manipulate result.

In [2]:
# Connect to database file sqlite
conn = sqlite3.connect('database.sqlite')

# and get information of all tables
tables = pd.read_sql("""
                        SELECT *
                          FROM sqlite_master
                         WHERE type = 'table'
                     """, conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


## Question 2: Select data in "Country" table

In [3]:
# select country_name  in "Country" table
pd.read_sql('''
SELECT name country_name
  FROM Country
            ''', conn)

,country_name
0,Belgium
1,England
2,France
3,Germany
4,Italy
5,Netherlands
6,Poland
7,Portugal
8,Scotland
9,Spain


## Question 3: Select data in "League" table

In [4]:
# select league_name  in "League" table

pd.read_sql('''
SELECT name league_name
  FROM League
            ''', conn)

,league_name
0,Belgium Jupiler League
1,England Premier League
2,France Ligue 1
3,Germany 1. Bundesliga
4,Italy Serie A
5,Netherlands Eredivisie
6,Poland Ekstraklasa
7,Portugal Liga ZON Sagres
8,Scotland Premier League
9,Spain LIGA BBVA


## Question 4: Select League data with country information
The League table has relation with Country table by country_id. Use the join sql statement to join two tables.

In [5]:
# The League table has relation with Country table by country_id

pd.read_sql('''
SELECT c.id, l.country_id, l.name league_name, c.name country_name 
  FROM Country c
    INNER JOIN League l 
    ON c.id = l.country_id
            ''', conn)

,id,country_id,league_name,country_name
0,1,1,Belgium Jupiler League,Belgium
1,1729,1729,England Premier League,England
2,4769,4769,France Ligue 1,France
3,7809,7809,Germany 1. Bundesliga,Germany
4,10257,10257,Italy Serie A,Italy
5,13274,13274,Netherlands Eredivisie,Netherlands
6,15722,15722,Poland Ekstraklasa,Poland
7,17642,17642,Portugal Liga ZON Sagres,Portugal
8,19694,19694,Scotland Premier League,Scotland
9,21518,21518,Spain LIGA BBVA,Spain


## Question 5: Select data from _Match_ table
- When a team is serving as host of a contest, it is designated as the "home team". The opposing team is said to be the "away 
team"
- In Match table, each row is a match with one home team and one away team including home team goal and away team goal respectively

In [6]:
# When a team is serving as host of a contest, it is designated as the "home team". The opposing team is said to be the "away team"
# In Match table, each row is a match with one home team and one away team including home team goal and away team goal respectively

pd.read_sql('''
                SELECT m.id, match_api_id, T1.team_long_name home_team_name, home_team_goal, away_team_goal, T2.team_long_name away_team_name
                FROM Match m 
                INNER JOIN Team T1 
                ON T1.team_api_id = m.home_team_api_id
                INNER JOIN Team T2 
                ON T2.team_api_id = m.away_team_api_id
            ''', conn)

,id,match_api_id,home_team_name,home_team_goal,away_team_goal,away_team_name
0,1,492473,KRC Genk,1,1,Beerschot AC
1,2,492474,SV Zulte-Waregem,0,0,Sporting Lokeren
2,3,492475,KSV Cercle Brugge,0,3,RSC Anderlecht
3,4,492476,KAA Gent,5,0,RAEC Mons
4,5,492477,FCV Dender EH,1,3,Standard de Liège
...,...,...,...,...,...,...
25974,25975,1992091,FC St. Gallen,1,0,FC Thun
25975,25976,1992092,FC Vaduz,1,2,FC Luzern
25976,25977,1992093,Grasshopper Club Zürich,2,0,FC Sion
25977,25978,1992094,Lugano,0,0,FC Zürich


## Question 6: Select match data includes league and country information

The Match table has relation with:
 - Country table by country_id
 - League table by league_id
 
Use the join sql statement to join 3 tables.


In [7]:
# Use the join sql statement to join 3 tables.
# Country table by country_id
# League table by league_id

pd.read_sql('''
SELECT m.id, match_api_id, home_team_api_id, away_team_api_id, l.name league_name, c.name country_name 
  FROM Match m
    INNER JOIN League l
    ON l.id = m.league_id
    INNER JOIN Country c 
    ON c.id = m.country_id
            ''', conn)

,id,match_api_id,home_team_api_id,away_team_api_id,league_name,country_name
0,1,492473,9987,9993,Belgium Jupiler League,Belgium
1,2,492474,10000,9994,Belgium Jupiler League,Belgium
2,3,492475,9984,8635,Belgium Jupiler League,Belgium
3,4,492476,9991,9998,Belgium Jupiler League,Belgium
4,5,492477,7947,9985,Belgium Jupiler League,Belgium
...,...,...,...,...,...,...
25974,25975,1992091,10190,10191,Switzerland Super League,Switzerland
25975,25976,1992092,9824,10199,Switzerland Super League,Switzerland
25976,25977,1992093,9956,10179,Switzerland Super League,Switzerland
25977,25978,1992094,7896,10243,Switzerland Super League,Switzerland


## Question 7: Get number of match by each league including league name, order the number of match by descending

In [8]:
# Dùng INNER JOIN với "League" và "Match" ,sau dó với COUNT(*) để truy vấn số lượng trận đấu, gán với tên number_of_matches
# Nhóm theo tên các giải đấu bằng GROUP BY và sau đó dùng ORDER BY để có giá trị giảm dần với DESC  

pd.read_sql('''
  SELECT l.name league_name, count(*) number_of_matches
    FROM Match m
        INNER JOIN League l 
        ON l.id = m.league_id
GROUP BY league_name
ORDER BY number_of_matches DESC
            ''', conn)

,league_name,number_of_matches
0,Spain LIGA BBVA,3040
1,France Ligue 1,3040
2,England Premier League,3040
3,Italy Serie A,3017
4,Netherlands Eredivisie,2448
5,Germany 1. Bundesliga,2448
6,Portugal Liga ZON Sagres,2052
7,Poland Ekstraklasa,1920
8,Scotland Premier League,1824
9,Belgium Jupiler League,1728


## Question 8: Get total goal of home team and away team in each league
- use Group By statement

In [9]:
# Use INNER JOIN with "League" and "Match" , then sum the goals of home team and away team with sum() and assign a new name
# Group by tournament name with GROUP BY for better classification

pd.read_sql('''
  SELECT l.name league_name, sum(home_team_goal) total_home_team_goal, sum(away_team_goal) total_away_team_goal
    FROM Match m
        INNER JOIN League l 
        ON l.id = m.league_id
GROUP BY league_name
            ''', conn)

,league_name,total_home_team_goal,total_away_team_goal
0,Belgium Jupiler League,2781,2060
1,England Premier League,4715,3525
2,France Ligue 1,4265,3162
3,Germany 1. Bundesliga,3982,3121
4,Italy Serie A,4528,3367
5,Netherlands Eredivisie,4357,3185
6,Poland Ekstraklasa,2678,1978
7,Portugal Liga ZON Sagres,2890,2311
8,Scotland Premier League,2607,2197
9,Spain LIGA BBVA,4959,3453


## Question 9: Select data from Team table

In [10]:
# Select data from Team table

pd.read_sql('''
SELECT * 
  FROM Team
            ''',conn)

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB
...,...,...,...,...,...
294,49479,10190,898.0,FC St. Gallen,GAL
295,49837,10191,1715.0,FC Thun,THU
296,50201,9777,324.0,Servette FC,SER
297,50204,7730,1862.0,FC Lausanne-Sports,LAU


## Question 10: Get top 20 teams with highest home goal
- Use GROUP BY, LIMIT statement

In [11]:
# The Match table is related to the team table via the keyword home_team_api_id
# Use GROUP BY, LIMIT statement

pd.read_sql('''
  SELECT Team_long_name, sum(home_team_goal) total_goals_in_home
    FROM Team T
        INNER JOIN Match m 
        ON m.home_team_api_id = T.team_api_id
GROUP BY T.team_long_name
ORDER BY total_goals_in_home DESC
   LIMIT 20
            ''',conn)

,team_long_name,total_goals_in_home
0,Real Madrid CF,505
1,FC Barcelona,495
2,Celtic,389
3,FC Bayern Munich,382
4,PSV,370
5,Manchester City,365
6,Ajax,360
7,FC Basel,344
8,Manchester United,338
9,Chelsea,333


## Question 11: Get top 20 teams with highest away goal
- Use GROUP BY, LIMIT statement

In [12]:
# The Match table is related to the team table via the keyword home_team_api_id
# Use GROUP BY, LIMIT statement
pd.read_sql('''
  SELECT Team_long_name, sum(away_team_goal) total_goals_in_away
    FROM Team T
        INNER JOIN Match m 
        ON m.home_team_api_id = T.team_api_id
GROUP BY T.team_long_name
ORDER BY total_goals_in_away DESC
   LIMIT 20
            ''',conn)

,team_long_name,total_goals_in_away
0,Kilmarnock,238
1,SV Werder Bremen,203
2,SC Heerenveen,202
3,Motherwell,201
4,ADO Den Haag,200
5,Aston Villa,198
6,VfB Stuttgart,195
7,Hannover 96,195
8,Dundee United,194
9,Willem II,192


## Question 12: Get team long name with total of goal, order the total number by descending

- Total of goal of a team is added up from both away and home games
- Use UNION statement 

In [13]:
# use UNION to merge 2 tables (home team code, home goals number) and (away team code, away team goals)

pd.read_sql('''
  SELECT T.team_long_name, sum(total_of_goal) total_of_goal
    FROM (
            SELECT home_team_api_id team_id, home_team_goal total_of_goal FROM Match

             UNION ALL

            SELECT away_team_api_id team_id, away_team_goal total_of_goal FROM Match) m
        INNER JOIN Team t  
        ON m.team_id = t.team_api_id
GROUP BY t.team_long_name
ORDER BY total_of_goal DESC
            ''',conn)

,team_long_name,total_of_goal
0,FC Barcelona,849
1,Real Madrid CF,843
2,Celtic,695
3,FC Bayern Munich,653
4,PSV,652
...,...,...
291,Amadora,26
292,Trofense,25
293,FC Dordrecht,24
294,Córdoba CF,22


## Question 13: Get team long name with total of matches, order the total number by descending

In [14]:
# Use UNION ALL to combine home_team and away_team names

pd.read_sql('''
  SELECT team_long_name, count(*) total_of_matches
    FROM (
        SELECT home_team_api_id team_api_id_name FROM Match 
        
        UNION ALL
        
        SELECT away_team_api_id team_api_id_name FROM Match) m
    INNER JOIN Team T
    ON m.team_api_id_name = T.team_api_id
GROUP BY team_long_name
ORDER BY total_of_matches DESC
            ''', conn)

,team_long_name,total_of_matches
0,Valencia CF,304
1,Toulouse FC,304
2,Tottenham Hotspur,304
3,Sunderland,304
4,Stoke City,304
...,...,...
291,Termalica Bruk-Bet Nieciecza,30
292,Portimonense,30
293,KAS Eupen,30
294,Feirense,30


## Question 14: Get numbers of win, lost and draw matches of each team

- If a team has home goal > away goal , team is “win” in this match
- If a team has home goal < away goal , team is “lost” in this match
- If a team has home goal = away goal , team is “draw” in this match
- Use Case When statement


In [15]:
# Use Case When statement

pd.read_sql('''
  SELECT team_long_name,
         (CASE
          WHEN our_team_goal > rival_team_goal THEN 'Win'
          WHEN our_team_goal = rival_team_goal THEN 'Draw'
          ELSE 'Lost'
           END ) outcome, count(*) quantity
    FROM (
        SELECT home_team_api_id team_id_name, home_team_goal our_team_goal, away_team_goal rival_team_goal FROM Match
        
         UNION ALL
         
        SELECT away_team_api_id team_id_name, away_team_goal our_team_goal, home_team_goal rival_team_goal FROM Match) m
    INNER JOIN Team T
    ON m.team_id_name = T.team_api_id
GROUP BY team_long_name, outcome
            ''', conn)

,team_long_name,outcome,quantity
0,1. FC Kaiserslautern,Draw,18
1,1. FC Kaiserslautern,Lost,33
2,1. FC Kaiserslautern,Win,17
3,1. FC Köln,Draw,54
4,1. FC Köln,Lost,90
...,...,...,...
883,Évian Thonon Gaillard FC,Lost,71
884,Évian Thonon Gaillard FC,Win,45
885,Śląsk Wrocław,Draw,80
886,Śląsk Wrocław,Lost,71


## Question 15: Get top 10 team with highest numbers of win matches

In [16]:
# Get top 10 team with highest numbers of win matches
# Using CASE WHEN , compare which team's goal is greater than the other's
# The win team will return a value of 1, losing is 0
# Use UNION ALL to not remove duplicate values
# For descending value we use ORDER BY ending with DESC

pd.read_sql('''
    SELECT team_long_name, SUM(
          CASE WHEN our_goal > rival_goal THEN 1 
          ELSE 0
           END ) number_of_win
      FROM (
            SELECT home_team_api_id team_id_name, home_team_goal our_goal, away_team_goal rival_goal FROM Match

             UNION ALL

            SELECT away_team_api_id team_id_name, away_team_goal our_goal, home_team_goal rival_goal FROM Match) m
        INNER JOIN Team T
        ON m.team_id_name = T.team_api_id
  GROUP BY team_long_name
  ORDER BY number_of_win DESC
     LIMIT 10
            ''', conn)

,team_long_name,number_of_win
0,FC Barcelona,234
1,Real Madrid CF,228
2,Celtic,218
3,FC Bayern Munich,193
4,Manchester United,192
5,Juventus,189
6,SL Benfica,185
7,FC Porto,183
8,Ajax,181
9,FC Basel,180


## Question 16: Get percentage of each league to join the matches

In [17]:
# Get percentage of each league to join the matches

pd.read_sql('''
  SELECT l.name league_name, ROUND(100.0* count(*)/(SELECT count(*) FROM Match), 1) percentage
    FROM Match m
        INNER JOIN League l 
        ON l.id = m.league_id
GROUP BY league_name
ORDER BY percentage DESC
            ''', conn)

,league_name,percentage
0,Spain LIGA BBVA,11.7
1,France Ligue 1,11.7
2,England Premier League,11.7
3,Italy Serie A,11.6
4,Netherlands Eredivisie,9.4
5,Germany 1. Bundesliga,9.4
6,Portugal Liga ZON Sagres,7.9
7,Poland Ekstraklasa,7.4
8,Scotland Premier League,7.0
9,Belgium Jupiler League,6.7


## Question 17: Get percentage of score in each league

In [18]:
# Get percentage of score in each league

pd.read_sql('''
  SELECT l.name league_name, ROUND (100.0 * (SUM(home_team_goal)+SUM(away_team_goal))/(SELECT SUM(home_team_goal)+SUM(away_team_goal) FROM Match), 2) per_of_score 
    FROM Match m
        INNER JOIN League l 
        ON l.id = m.league_id
GROUP BY league_name
            ''', conn)

,league_name,per_of_score
0,Belgium Jupiler League,6.89
1,England Premier League,11.72
2,France Ligue 1,10.57
3,Germany 1. Bundesliga,10.11
4,Italy Serie A,11.23
5,Netherlands Eredivisie,10.73
6,Poland Ekstraklasa,6.62
7,Portugal Liga ZON Sagres,7.40
8,Scotland Premier League,6.83
9,Spain LIGA BBVA,11.97


## Question 18: Get total numbers of goals for each league in each season

In [19]:
# Get total numbers of goals for each league in each season

pd.read_sql('''
  SELECT l.name league_name, season, (SUM(home_team_goal) + SUM(away_team_goal)) numbers_of_goals
    FROM Match m
        INNER JOIN League l 
        ON l.id = m.league_id
GROUP BY league_name, season
            ''', conn)

,league_name,season,numbers_of_goals
0,Belgium Jupiler League,2008/2009,855
1,Belgium Jupiler League,2009/2010,565
2,Belgium Jupiler League,2010/2011,635
3,Belgium Jupiler League,2011/2012,691
4,Belgium Jupiler League,2012/2013,703
...,...,...,...
83,Switzerland Super League,2011/2012,425
84,Switzerland Super League,2012/2013,462
85,Switzerland Super League,2013/2014,520
86,Switzerland Super League,2014/2015,517


## Question 19: Get player attributes

- Convert weight to kilogram
- Convert height to meter
- Calculuate bmi = ( (weight* 0.453592) / (height/100)^2)
- Get Age of player

In [20]:
# Use ROUND() to round to 2 decimals
# Use strftime() to calculate the age of the players

pd.read_sql('''
SELECT player_name, 
    ROUND (weight * 0.453592, 2) weight_kg, 
    ROUND (height/100, 2) height_m,
    ROUND ((weight * 0.453592) / (height*height/10000), 2) BMI, 
    strftime('%Y', date('now')) - strftime('%Y', date(birthday)) years_old
  FROM Player
            ''',conn)

,player_name,weight_kg,height_m,BMI,years_old
0,Aaron Appindangoye,84.82,1.83,25.36,30
1,Aaron Cresswell,66.22,1.70,22.87,33
2,Aaron Doran,73.94,1.70,25.53,31
3,Aaron Galindo,89.81,1.83,26.85,40
4,Aaron Hughes,69.85,1.83,20.89,43
...,...,...,...,...,...
11055,Zoumana Camara,76.20,1.83,22.78,43
11056,Zsolt Laczko,79.83,1.83,23.87,36
11057,Zsolt Low,69.85,1.80,21.48,43
11058,Zurab Khizanishvili,78.02,1.85,22.69,41


## Question 20: Get oldest player

In [21]:
# Get oldest player
# Use strftime() to calculate the age of the players

pd.read_sql('''
  SELECT player_name, strftime('%Y', date('now')) - strftime('%Y', birthday) years_old
    FROM Player
ORDER BY years_old DESC
   LIMIT 1
            ''',conn)

,player_name,years_old
0,Alberto Fontana,55


## Question 21: Get player who played highest number of matches

In [22]:
# The Player table has relation with Player_Attributes table by player_api_id
# Assuming each row of data is a match, to find the total number of times played by each player we use COUNT()
# Sorting by group of player names and using ORDER BY() with decreasing number, we will find the player with the most matches.
# Below is the list of players and their match count, use LIMIT to limit the above list as required

pd.read_sql('''
  SELECT P.player_name, count(*) number_of_matches
    FROM Player_Attributes Pa
        INNER JOIN Player P
        ON P.player_api_id = Pa.player_api_id
GROUP BY P.player_name
ORDER BY number_of_matches DESC
   LIMIT 1
            ''',conn)

,player_name,number_of_matches
0,Danilo,108


## Question 22: Get players who had overall_rating larger than 80

In [23]:
# The Player table has relation with Player_Attributes table by player_api_id
# Each player has multiple rating points, using AVG() to average the rating greater than 80, use ROUND() to round 2 decimals.
# When the query returns a match for the group, we use HAVING() after GROUP BY().

pd.read_sql('''
  SELECT P.player_name, ROUND(AVG(overall_rating), 2) avg_of_overall_rating
    FROM Player_Attributes Pa
        INNER JOIN Player P
        ON P.player_api_id = Pa.player_api_id
GROUP BY P.player_name
  HAVING avg_of_overall_rating> 80
            ''',conn)

,player_name,avg_of_overall_rating
0,"Afonso Alves,24",80.29
1,Alessandro Nesta,82.16
2,Alexandre Pato,81.13
3,Alexis Sanchez,82.57
4,Alvaro Negredo,81.27
...,...,...
200,Xabi Alonso,84.54
201,Xavi Hernandez,87.64
202,Yaya Toure,85.03
203,Yevheniy Konoplyanka,82.00
